In [17]:
# pip install --upgrade protobuf
# protoc --python_out=. your_proto_file.proto
# pip install protobuf==3.20
# !pip install --upgrade tensorflow
!pip install tensorflow


  Using cached tensorflow-2.12.0-cp39-cp39-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.0-cp39-cp39-win_amd64.whl (272.8 MB)
  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.13.0
    Uninstalling tensorboard-2.13.0:
      Successfully uninstalled tensorboard-2.13.0


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\MSI\\anaconda3\\Lib\\site-packages\\tensorflow\\python\\_pywrap_tensorflow_internal.pyd'
Consider using the `--user` option or check the permissions.



In [10]:
import sys
print(sys.version)

3.9.16 (main, Mar  1 2023, 18:30:21) [MSC v.1916 64 bit (AMD64)]


In [11]:
from keras.layers.convolutional import Conv3D, ZeroPadding3D
from keras.layers.pooling import MaxPooling3D
from keras.layers.core import Dense, Activation, Dropout, Flatten, SpatialDropout3D
from keras.layers import Bidirectional, TimeDistributed
from keras.layers import BatchNormalization
#from keras.layers.recurrent import GRU
from keras.layers import GRU
from keras.layers import Input
from keras.models import Model
from keras import backend as K
from keras.layers.core import Lambda
from keras import backend as K
import tensorflow as tf

In [12]:
from keras.layers.core import Lambda
from keras import backend as K

# Actual loss calculation
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # From Keras example image_ocr.py:
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    # y_pred = y_pred[:, 2:, :]
    y_pred = y_pred[:, :, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def CTC(name, args):
    return Lambda(ctc_lambda_func, output_shape=(1,), name=name)(args)

In [13]:
input_shape = (5, 100, 200, 3)
output_size = 13
x = Input(name = 'the_input', shape = input_shape, dtype = 'float32')
print(x.shape)
x = ZeroPadding3D(padding = (1,2,2), name = 'zero1')(x)
print(x.shape)
x = Conv3D(32, (3,5,5), strides = (1,2,2), activation = 'relu', kernel_initializer = 'he_normal', name = 'conv1')(x)
print(x.shape)
x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max1')(x)
print(x.shape)
x = Dropout(0.5)(x)
print(x.shape)
print("=====================================================")
x = ZeroPadding3D(padding=(1, 2, 2), name='zero2')(x)
print(x.shape)
x = Conv3D(64, (3, 5, 5), strides=(1, 1, 1), activation='relu', kernel_initializer='he_normal', name='conv2')(x)
print(x.shape)
x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max2')(x)
print(x.shape)
x = Dropout(0.5)(x)
print(x.shape)
print("=====================================================")
x = ZeroPadding3D(padding=(1, 1, 1), name='zero3')(x)
print(x.shape)
x = Conv3D(96, (3, 3, 3), strides=(1, 1, 1), activation='relu', kernel_initializer='he_normal', name='conv3')(x)
print(x.shape)
x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max3')(x)
print(x.shape)
x = Dropout(0.5)(x)
print(x.shape) # (BN, 5, 6, 12, 96)
print("=====================================================")
x = TimeDistributed(Flatten())(x)
print(x.shape) # (BN, 5, 6912)
print("=====================================================")
x = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru1'), merge_mode='concat')(x)
print(x.shape)
x = Bidirectional(GRU(256, return_sequences=False, kernel_initializer='Orthogonal', name='gru2'), merge_mode='concat')(x)
print(x.shape)
print("=====================================================")
x = Dense(output_size, kernel_initializer='he_normal', name='dense1')(x)
print(x.shape)
print("=====================================================")
y_pred = Activation('softmax', name='softmax')(x)
print(y_pred.shape)

(None, 5, 100, 200, 3)
(None, 7, 104, 204, 3)
(None, 5, 50, 100, 32)
(None, 5, 25, 50, 32)
(None, 5, 25, 50, 32)
(None, 7, 29, 54, 32)
(None, 5, 25, 50, 64)
(None, 5, 12, 25, 64)
(None, 5, 12, 25, 64)
(None, 7, 14, 27, 64)
(None, 5, 12, 25, 96)
(None, 5, 6, 12, 96)
(None, 5, 6, 12, 96)
(None, 5, 6912)
(None, 5, 512)
(None, 512)
(None, 13)
(None, 13)


In [14]:
class LipNet(object):
    def __init__(self, img_c, img_w, img_h, frames_n, output_size):
        self.img_c = img_c
        self.img_w = img_w
        self.img_h = img_h
        self.frames_n = frames_n
        self.output_size = output_size
        self.build()
        
    def build(self):
        input_shape = (self.frames_n, self.img_w, self.img_h, self.img_c)
        #if K.image_data_format() == 'channels_first':
            #input_shape = (self.img_c, self.frames_n, self.img_w, self.img_h)
        #else:
            #input_shape = (self.frames_n, self.img_w, self.img_h, self.img_c)
        
        self.input_data = Input(name = 'the_input', shape = (5, 100, 200, 3), dtype = 'float32')
        self.zero1 = ZeroPadding3D(padding=(1, 2, 2), name='zero1')(self.input_data)
        self.conv1 = Conv3D(32, (3, 5, 5), strides=(1, 2, 2), kernel_initializer='he_normal', name='conv1')(self.zero1)
        self.batc1 = BatchNormalization(name='batc1')(self.conv1)
        self.actv1 = Activation('relu', name='actv1')(self.batc1)
        self.drop1 = SpatialDropout3D(0.5)(self.actv1)
        self.maxp1 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max1')(self.drop1)

        self.zero2 = ZeroPadding3D(padding=(1, 2, 2), name='zero2')(self.maxp1)
        self.conv2 = Conv3D(64, (3, 5, 5), strides=(1, 1, 1), kernel_initializer='he_normal', name='conv2')(self.zero2)
        self.batc2 = BatchNormalization(name='batc2')(self.conv2)
        self.actv2 = Activation('relu', name='actv2')(self.batc2)
        self.drop2 = SpatialDropout3D(0.5)(self.actv2)
        self.maxp2 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max2')(self.drop2)

        self.zero3 = ZeroPadding3D(padding=(1, 1, 1), name='zero3')(self.maxp2)
        self.conv3 = Conv3D(96, (3, 3, 3), strides=(1, 1, 1), kernel_initializer='he_normal', name='conv3')(self.zero3)
        self.batc3 = BatchNormalization(name='batc3')(self.conv3)
        self.actv3 = Activation('relu', name='actv3')(self.batc3)
        self.drop3 = SpatialDropout3D(0.5)(self.actv3)
        self.maxp3 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max3')(self.drop3)


        self.resh1 = TimeDistributed(Flatten())(self.maxp3)


        self.gru_1 = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru1'), merge_mode='concat')(self.resh1)
        self.gru_2 = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru2'), merge_mode='concat')(self.gru_1)

        # transforms RNN output to character activations:
        self.dense1 = Dense(self.output_size, kernel_initializer='he_normal', name='dense1')(self.gru_2)

        self.y_pred = Activation('softmax', name='softmax')(self.dense1)
        
        ####
        # y_true: (samples, max_string_length) : containing the truth labels
        # y_pred: (samples, time_steps, num_categories) : containing the prediction, or output of the softmax
        # input_length: (samples, 1): containing the sequence length for each batch item in y_pred
        # label_length: (samples, 1): containing the sequence length for each batch item in y_true
        ####
        self.labels = Input(name='the_labels', shape=[13], dtype='float32')
        self.input_length = Input(name='input_length', shape=[1], dtype='int64')
        self.label_length = Input(name='label_length', shape=[1], dtype='int64')

        self.loss_out = CTC('ctc', [self.y_pred, self.labels, self.input_length, self.label_length])

        self.model = Model(inputs=[self.input_data, self.labels, self.input_length, self.label_length], outputs=self.loss_out)
        
    def summary(self):
        Model(inputs = self.input_data, outputs = self.y_pred).summary()
        
    def predict(self, input_batch):
        return self.test_function([input_batch, 0])[0]  # the first 0 indicates test
    
    @property
    def test_function(self):
        # captures output of softmax so we can decode the output during visualization
        return K.function([self.input_data, K.learning_phase()], [self.y_pred, K.learning_phase()])

In [21]:
from tensorflow.keras.optimizers import legacy

lipnet = LipNet(img_c=3, img_w=100, img_h=200, frames_n=5, output_size=13)
lipnet.summary()
adam = legacy.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
lipnet.model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam, metrics=['accuracy']) 

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 the_input (InputLayer)      [(None, 5, 100, 200, 3)]  0         
                                                                 
 zero1 (ZeroPadding3D)       (None, 7, 104, 204, 3)    0         
                                                                 
 conv1 (Conv3D)              (None, 5, 50, 100, 32)    7232      
                                                                 
 batc1 (BatchNormalization)  (None, 5, 50, 100, 32)    128       
                                                                 
 actv1 (Activation)          (None, 5, 50, 100, 32)    0         
                                                                 
 spatial_dropout3d_12 (Spati  (None, 5, 50, 100, 32)   0         
 alDropout3D)                                                    
                                                           

In [22]:
from keras.models import load_model
model = lipnet.model.load_weights("ModelCheckpoint/")

In [23]:
model

In [24]:
import dlib
import cv2
import numpy as np
import tensorflow as tf
from keras.preprocessing import image

In [25]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((20, 2), dtype=dtype)
    # for only lip landmarks
    n = 0
    for i in range(48, shape.num_parts):
        coords[n] = (shape.part(i).x, shape.part(i).y)
        n += 1

    # return the list of (x, y)-coordinates
    return coords

In [26]:
def get_rect(shape):
    rw = 0
    rh = 0
    rx = 65535
    ry = 65535
    for (x,y) in shape:
        rw = max(rw,x) # w
        rh = max(rh,y) # h
        rx = min(rx,x) # x
        ry = min(ry,y) # y
    return (rx,ry,rw-rx,rh-ry)

In [28]:
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("data/shape_predictor_68_face_landmarks.dat") # predict 해주는애       

count = 0

cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        image = cv2.resize(frame, dsize = (640, 480), interpolation = cv2.INTER_AREA)
        img = image.copy()
        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        face_detector = detector(img_gray, 1)
        cv2.imshow('frame',image)  
        for face in face_detector:
            landmarks = predictor(image, face)
            landmarks = shape_to_np(landmarks)
            # select center of mouth
            x_list = [x[0] for x in landmarks]
            y_list = [y[1] for y in landmarks]
            x = sum(x_list)//20
            y = sum(y_list)//20
            dst = img[y-50:y+50, x-100:x+100].copy()
            img_tensor = img_to_array(dst) #tensor로 type 변경
            img_tensor /= 255. # scaling  # (100, 200, 3)
            img_tensor = np.expand_dims(img_tensor, axis = 0) #(1, 100, 200, 3)  
            # frame 별 묶기
            if count == 0:
                x_data = img_tensor
                count += 1
            else:
                x_data = np.concatenate((x_data, img_tensor), axis = 0)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

In [29]:
x_data[:5].shape

(5, 100, 200, 3)

In [30]:
data = x_data[:5].reshape(-1,5,100, 200, 3)

In [31]:
print(data.shape)

(1, 5, 100, 200, 3)


In [ ]:
answer = model.predict(data)
print(answer)